## Data Modelling

#### Required fields
* Transaction type (i.e. sale vs. rent - string)
* Bedrooms (integer)
* Bathrooms (integer)
* Description (free text string)
* Property type e.g. flat, detached house, terraced house
* Price e.g. 500,000 (typically integer)
* Location :  Key location data here is Postcode district and/or Postcode
* Agent (advertising the property)
* Listing source
* Listing URL
* Other nice-to-have metadata
* If a rental property is furnished or not
* Anything else you deem interesting

In [1]:
# import libaries needed
import pandas as pd
import psycopg2
from datetime import datetime, date
import csv


In [2]:
# load dataset
rm_data = pd.read_csv(f'data_output/rightnow_{date.today()}.csv')
omt_data = pd.read_csv(f'data_output/omt_{date.today()}.csv')
z_data = pd.read_csv(f'data_output/zoopla_{date.today()}.csv')

In [3]:
rm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction     2100 non-null   object 
 1   address         2100 non-null   object 
 2   bedroom         1968 non-null   float64
 3   bathroom        1953 non-null   float64
 4   sales_price     1045 non-null   float64
 5   rent_perMonth   1050 non-null   float64
 6   rent_perWeek    1050 non-null   float64
 7   description     2100 non-null   object 
 8   propertyType    2100 non-null   object 
 9   location        2100 non-null   object 
 10  agent           2100 non-null   object 
 11  listing_source  2100 non-null   object 
 12  listing_url     2100 non-null   object 
 13  listed_date     1386 non-null   object 
dtypes: float64(5), object(9)
memory usage: 229.8+ KB


In [4]:
omt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1886 entries, 0 to 1885
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction     1886 non-null   object 
 1   address         1804 non-null   object 
 2   bedroom         1686 non-null   float64
 3   bathroom        1718 non-null   float64
 4   sales_price     861 non-null    float64
 5   rent_perMonth   943 non-null    float64
 6   rent_perWeek    943 non-null    float64
 7   description     1804 non-null   object 
 8   propertyType    1768 non-null   object 
 9   location        1804 non-null   object 
 10  agent           164 non-null    object 
 11  listing_source  1886 non-null   object 
 12  listing_url     1886 non-null   object 
 13  listed_date     1804 non-null   object 
dtypes: float64(5), object(9)
memory usage: 206.4+ KB


In [5]:
z_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction     2100 non-null   object 
 1   address         2100 non-null   object 
 2   bedroom         2058 non-null   float64
 3   bathroom        2018 non-null   float64
 4   living_room     1583 non-null   float64
 5   sales_price     1050 non-null   float64
 6   rent_perMonth   1050 non-null   float64
 7   rent_perWeek    1050 non-null   float64
 8   description     2100 non-null   object 
 9   propertyType    1975 non-null   object 
 10  location        2100 non-null   object 
 11  agent           2100 non-null   object 
 12  listing_source  2100 non-null   object 
 13  listing_url     2100 non-null   object 
 14  listed_date     2100 non-null   object 
dtypes: float64(6), object(9)
memory usage: 246.2+ KB


### Create a Unique ID for each data

The unique id will be crated by combining the  [[transaction,agent,address,sales_price,rent_permonth,rent_perweek]] as one attribute and drop all empty columns

In [6]:
# create a unique id field
rm_data['id'] = rm_data['transaction'] + rm_data["bedroom"].astype(str) + rm_data["bathroom"].astype(str) + rm_data["sales_price"].astype(str) + z_data["rent_perMonth"].astype(str) + rm_data["address"]
# drop empty columns
rm_data.dropna(subset=['address'], inplace=True)

# drop duplicate id
rm_data.drop_duplicates(subset="id" , inplace=True)

# read dataset
rm_data.head()

,transaction,address,bedroom,bathroom,sales_price,rent_perMonth,rent_perWeek,description,propertyType,location,agent,listing_source,listing_url,listed_date,id
0,rent,"Marcon Place, London, E8 1LP, UK",2.0,2.0,NaN,3230.0,745.0,Show up and start living from day one in Londo...,Apartment,UK,"BLUEGROUND FURNISHED APARTMENTS UK LTD, London",rightmove,https://www.rightmove.co.uk/properties/1318127...,2023-05-20,"rent2.02.0nan1350.0Marcon Place, London, E8 1L..."
1,rent,"Powis Gardens, London, NW11",NaN,1.0,NaN,1200.0,277.0,Property Reference Number: 100684. This first ...,Apartment,NW11,"PropertyLoop, London",rightmove,https://www.rightmove.co.uk/properties/1350657...,2023-05-20,"rentnan1.0nan1400.0Powis Gardens, London, NW11"
2,rent,"Fitzjohn's Avenue, Hampstead, NW3",3.0,3.0,NaN,8498.0,1961.0,"Forming part of this magnificent development, ...",Flat,NW3,"TK International, Hampstead",rightmove,https://www.rightmove.co.uk/properties/1350656...,2023-05-20,"rent3.03.0nan1450.0Fitzjohn's Avenue, Hampstea..."
3,rent,"Loddiges Road, Hackney, London, E9 6PR",3.0,1.0,NaN,2817.0,650.0,Bryan Estates are pleased to present this spac...,Maisonette,6PR,"Bryan Estates, London",rightmove,https://www.rightmove.co.uk/properties/1350656...,2023-05-20,"rent3.01.0nan1300.0Loddiges Road, Hackney, Lon..."
4,rent,"Chelmsford House, London, N7",1.0,1.0,NaN,1700.0,392.0,One bed flat (400 square feet) with separate k...,Flat,N7,"OpenRent, London",rightmove,https://www.rightmove.co.uk/properties/1350646...,2023-05-20,"rent1.01.0nan3792.0Chelmsford House, London, N7"


In [7]:
# create a unique id field
omt_data['id'] = omt_data['id'] = omt_data['transaction'] + omt_data["bedroom"].astype(str) + omt_data["bathroom"].astype(str) + omt_data["sales_price"].astype(str) + z_data["rent_perMonth"].astype(str) + omt_data["address"]
# drop empty columns
omt_data.dropna(subset=['address'], inplace=True)

# drop duplicate id
omt_data.drop_duplicates(subset="id", inplace=True)

# read dataset
omt_data.head()

,transaction,address,bedroom,bathroom,sales_price,rent_perMonth,rent_perWeek,description,propertyType,location,agent,listing_source,listing_url,listed_date,id
0,rent,"Islington On The Green, 12A Islington Green, A...",1.0,1.0,NaN,2665.0,615.0,1 bedroom apartment to rent,apartment,N1,Savills - Islington,omt,https://www.onthemarket.com/details/776007/,> 14 days,"rent1.01.0nan1350.0Islington On The Green, 12A..."
1,rent,"St Georges Square, Pimlico, SW1V 3QX",2.0,1.0,NaN,2800.0,646.0,2 bedroom flat to rent,flat,3QX,Tennison Property - Wimbledon,omt,https://www.onthemarket.com/details/13234905/,2023-05-20,"rent2.01.0nan1400.0St Georges Square, Pimlico,..."
2,rent,"Bushey Road, Raynes Park, London, SW20 0JF",1.0,1.0,NaN,1495.0,345.0,1 bedroom flat to rent,flat,0JF,NaN,omt,https://www.onthemarket.com/details/13205276/,2023-05-20,"rent1.01.0nan1450.0Bushey Road, Raynes Park, L..."
3,rent,"Lascotts Road, Wood Green, London, N22 8JG",2.0,1.0,NaN,1647.0,380.0,2 bedroom flat to rent,flat,8JG,NaN,omt,https://www.onthemarket.com/details/13234903/,2023-05-20,"rent2.01.0nan1300.0Lascotts Road, Wood Green, ..."
4,rent,"Loddiges Road, Hackney, London, E9 6PR",3.0,1.0,NaN,2817.0,650.0,3 bedroom maisonette to rent,maisonette,6PR,NaN,omt,https://www.onthemarket.com/details/13234904/,2023-05-20,"rent3.01.0nan3792.0Loddiges Road, Hackney, Lon..."


In [8]:
# create a unique id field
z_data['id'] = z_data['id'] = z_data['transaction'] + z_data["bedroom"].astype(str) + z_data["bathroom"].astype(str) + z_data["sales_price"].astype(str) + z_data["rent_perMonth"].astype(str) + z_data["address"]
# drop empty columns
z_data.dropna(subset=['address'], inplace=True)

# drop duplicate id
z_data.drop_duplicates(subset="id", inplace=True)

# read dataset
z_data.head()

,transaction,address,bedroom,bathroom,living_room,sales_price,rent_perMonth,rent_perWeek,description,propertyType,location,agent,listing_source,listing_url,listed_date,id
0,rent,"Woodstock Gardens, Beckenham BR3",1.0,1.0,1.0,NaN,1350.0,312.0,Fantastic 1 bedroom ground floor flat availabl...,flat,BR3,Andrew Reeves Countrywide - Beckenham,zoopla,https://www.zoopla.co.uk/to-rent/details/54145...,20-05-2023,"rent1.01.0nan1350.0Woodstock Gardens, Beckenha..."
1,rent,"238 London Road, Twickenham TW1",1.0,1.0,1.0,NaN,1400.0,323.0,*** No Administration Fee if you move by 30th ...,NaN,TW1,John D Wood & Co. - St. Margarets Lettings,zoopla,https://www.zoopla.co.uk/to-rent/details/64676...,20-05-2023,"rent1.01.0nan1400.0238 London Road, Twickenham..."
2,rent,"Sandford Road, Bromley BR2",2.0,1.0,1.0,NaN,1450.0,335.0,**available in now** - two bedroom first floor...,NaN,BR2,Andrew Reeves Countrywide - Bromley,zoopla,https://www.zoopla.co.uk/to-rent/details/64676...,20-05-2023,"rent2.01.0nan1450.0Sandford Road, Bromley BR2"
3,rent,"London Road, Forest Hill SE23",1.0,1.0,1.0,NaN,1300.0,300.0,Short term - Commuters delight! This sassy one...,flat,SE23,Leaders - Forest Hill,zoopla,https://www.zoopla.co.uk/to-rent/details/64676...,20-05-2023,"rent1.01.0nan1300.0London Road, Forest Hill SE23"
4,rent,"City Road, London EC1V",2.0,1.0,NaN,NaN,3792.0,875.0,*Brand New* 2 bed 1 bath apartment to rent at ...,flat,EC1V,One London,zoopla,https://www.zoopla.co.uk/to-rent/details/64676...,20-05-2023,"rent2.01.0nan3792.0City Road, London EC1V"


### Connecting to the database

In [9]:

def create_database(database_name: str):
    # Connect to the default database
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=1118")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    # Create the sparkify database with UTF8 encoding
    cur.execute(f"DROP DATABASE IF EXISTS {database_name}")
    cur.execute(f"CREATE DATABASE {database_name} ENCODING 'UTF8'")

    # Close the connection to the default database
    cur.close()
    conn.close()

    # Connect to the new database
    conn = psycopg2.connect(f"host=localhost dbname={database_name} user=postgres password=1118")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    return cur, conn

cur, conn = create_database('london_propertylisting')

### Create table for each dataset and insert the data into the table
float is used instead of integer because there are NAN values which not be represented as integer

Right move

In [10]:
print(pd.io.sql.get_schema(rm_data, name="rm_data"))

CREATE TABLE "rm_data" (
"transaction" TEXT,
  "address" TEXT,
  "bedroom" REAL,
  "bathroom" REAL,
  "sales_price" REAL,
  "rent_perMonth" REAL,
  "rent_perWeek" REAL,
  "description" TEXT,
  "propertyType" TEXT,
  "location" TEXT,
  "agent" TEXT,
  "listing_source" TEXT,
  "listing_url" TEXT,
  "listed_date" TEXT,
  "id" TEXT
)


In [11]:
cur.execute("DROP TABLE IF EXISTS rightmove")

rm_table = ("""CREATE TABLE IF NOT EXISTS rightmove (
transaction TEXT,
  address VARCHAR,
  bedroom FLOAT,
  bathroom FLOAT,
  sales_price FLOAT,
  rent_perMonth FLOAT,
  rent_perWeek FLOAT,
  description VARCHAR,
  propertyType VARCHAR,
  location VARCHAR,
  agent VARCHAR,
  listing_source VARCHAR,
  listing_url VARCHAR,
  listed_date VARCHAR,
  id VARCHAR PRIMARY KEY
  );""")

cur.execute(rm_table)

In [12]:
# insert rm_data into the table created
rm_insert = """
INSERT INTO rightmove (
    transaction,
    address,
    bedroom,
    bathroom,
    sales_price,
    rent_perMonth,
    rent_perWeek,
    description,
    propertyType,
    location,
    agent,
    listing_source,
    listing_url,
    listed_date,
    id)
VALUES
(%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
for i, row in rm_data.iterrows():
    cur.execute(rm_insert, list(row))

On the market

In [13]:
cur.execute("DROP TABLE IF EXISTS omt")

omt_table = ("""CREATE TABLE IF NOT EXISTS omt (
transaction TEXT,
  address VARCHAR,
  bedroom FLOAT,
  bathroom FLOAT,
  sales_price FLOAT,
  rent_perMonth FLOAT,
  rent_perWeek FLOAT,
  description VARCHAR,
  propertyType VARCHAR,
  location VARCHAR,
  agent VARCHAR,
  listing_source VARCHAR,
  listing_url VARCHAR,
  listed_date VARCHAR,
  id VARCHAR PRIMARY KEY
  );""")

cur.execute(omt_table)

In [14]:
# insert omt_data into the table created
omt_insert = """
INSERT INTO omt (
    transaction,
    address,
    bedroom,
    bathroom,
    sales_price,
    rent_perMonth,
    rent_perWeek,
    description,
    propertyType,
    location,
    agent,
    listing_source,
    listing_url,
    listed_date,
    id)
VALUES
(%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
for i, row in omt_data.iterrows():
    cur.execute(omt_insert, list(row))

Zoopla

In [15]:
cur.execute("DROP TABLE IF EXISTS zoopla")

z_table = ("""CREATE TABLE IF NOT EXISTS zoopla (
transaction TEXT,
  address VARCHAR,
  bedroom FLOAT,
  bathroom FLOAT,
  living_room FLOAT,
  sales_price FLOAT,
  rent_perMonth FLOAT,
  rent_perWeek FLOAT,
  description VARCHAR,
  propertyType VARCHAR,
  location VARCHAR,
  agent VARCHAR,
  listing_source VARCHAR,
  listing_url VARCHAR,
  listed_date VARCHAR,
  id VARCHAR PRIMARY KEY
  );""")

cur.execute(z_table)

In [16]:
# insert rm_data into the table created
z_insert = """
INSERT INTO zoopla (
    transaction,
    address,
    bedroom,
    bathroom,
    living_room,
    sales_price,
    rent_perMonth,
    rent_perWeek,
    description,
    propertyType,
    location,
    agent,
    listing_source,
    listing_url,
    listed_date,
    id)
VALUES
(%s,%s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
for i, row in z_data.iterrows():
    cur.execute(z_insert, list(row))

### Pull out the dataset and write query

  Use LEFT outer join to combine all the three dataset as one, remove duplicate and save the distinct data set into a new table in the database and also ouput it as csv

In [17]:
query1 = """SELECT DISTINCT *
FROM (
    SELECT *
    FROM public.rightmove
    UNION ALL
    SELECT *
    FROM public.omt
    UNION ALL
    SELECT *
    FROM public.zoopla
) AS master_list;"""

In [18]:
query1 = """
SELECT DISTINCT *
FROM (
    SELECT transaction, address, bedroom, bathroom, sales_price, rent_perMonth, rent_perWeek,
        description, propertyType, location, agent, listing_source, listing_url, listed_date, id,
        CAST(NULL AS FLOAT) AS living_room
    FROM public.rightmove
    UNION ALL
    SELECT transaction, address, bedroom, bathroom, sales_price, rent_perMonth, rent_perWeek,
        description, propertyType, location, agent, listing_source, listing_url, listed_date, id,
        CAST(NULL AS FLOAT) AS living_room
    FROM public.omt
    UNION ALL
    SELECT transaction, address, bedroom, bathroom, sales_price, rent_perMonth, rent_perWeek,
        description, propertyType, location, agent, listing_source, listing_url, listed_date, id, living_room
    FROM public.zoopla
) AS master_list;
"""

In [19]:
cur.execute(query1)

for row in cur.fetchall():
    print(row)

('rent', 'Lees Road, Uxbridge, Middlesex, UB8', nan, 1.0, nan, 1200.0, 277.0, 'Studio to rent', 'Studio', 'UB8', 'NaN', 'omt', 'https://www.onthemarket.com/details/5865546/', '2023-05-19', 'rentnan1.0nan3500.0Lees Road, Uxbridge, Middlesex, UB8', None)
('rent', 'Aerial Square, London, NW9', 3.0, 2.0, nan, 2800.0, 646.0, 'A selection of newly built, modern and spacious studios, 1-, 2- & 3-bedroom apartments with access to a wide range of communal spaces. Easy transport links to Kings Cross within 22 minutes via Colindale station which is a 1-minute walk from UNCLE Colindale.', 'Apartment', 'NW9', 'Uncle, Colindale', 'rightmove', 'https://www.rightmove.co.uk/properties/135020942#/?channel=RES_LET', 'NaN', 'rent3.02.0nan2925.0Aerial Square, London, NW9', None)
('rent', 'Woolwich Arsenal, SE18', 2.0, nan, nan, 2000.0, 462.0, '**NO DEPOSIT OPTION AVAILABLE** - AMAZING, STYLISH, 2 DOUBLE BEDROOM SPLIT LEVEL APARTMENT WITH 2 BATHROOMS! Perfect apartment for commuters, only a short walk from W

### convert the query  resutl to csv and xlsx file

In [20]:
# # excel framework
# !pip install openpyxl

In [29]:
cur.execute(query1)
rows = cur.fetchall()

# Convert the query result to a DataFrame
df_1 = pd.DataFrame(rows, columns=[column[0] for column in cur.description])


df_1.drop_duplicates(subset="id", inplace= True)

std_order = [
    'transaction', 'address', 'bedroom', 'bathroom',  'living_room', 'sales_price', 'rent_permonth', 'rent_perweek',
    'description', 'propertytype', 'location', 'agent', 'listing_source', 'listing_url',
    'listed_date', 'id'
]

df_2 = df_1[std_order]

# Save as CSV
csv_filename = f'data_output/masterList_{date.today()}.csv'
df_2.to_csv(csv_filename, index=False)

# Save as Excel
excel_filename = f'data_output/masterList_{date.today()}.xlsx'
df_2.to_excel(excel_filename, index=False)

### Creare a database and load the master list back to the database

In [30]:
cur.execute("DROP TABLE IF EXISTS master_list")

master_list  = """
CREATE TABLE IF NOT EXISTS master_list (
    transaction TEXT,
    address VARCHAR,
    bedroom FLOAT,
    bathroom FLOAT,
    living_room FLOAT,
    sales_price FLOAT,
    rent_perMonth FLOAT,
    rent_perWeek FLOAT,
    description VARCHAR,
    propertyType VARCHAR,
    location VARCHAR,
    agent VARCHAR,
    listing_source VARCHAR,
    listing_url VARCHAR,
    listed_date VARCHAR,
    id VARCHAR PRIMARY KEY
    
);
"""

cur.execute(master_list)

Copy the saved csv file to the master_list table

In [31]:
file_path = f'D:\Data_Project_Portfolio\Mappa Creating a Master Property List for London\data_output/masterList_{date.today()}.csv'

try:
    cur.execute(f""" COPY master_list(transaction, address, bedroom, bathroom,living_room, sales_price, rent_perMonth, rent_perWeek,
    description, propertyType, location, agent, listing_source, 
    listing_url, listed_date, id)

    FROM '{file_path}'
    DELIMITER ','
    CSV HEADER;
    """)
    print("Success")
    
except psycopg2.Error as e:
    print("failed")
    print(e)

Success
